# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set the path to citis.csv
cities_path = "../output_data/cities.csv"

# Read the cities.csv
cities_df = pd.read_csv(cities_path)

# # Set the index to the City ID
cities_df = cities_df.set_index('City ID')

# output cities_df
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Dikson,98,RU,1603137316,95,73.51,80.55,23.77,17.83
1,Cabo San Lucas,5,MX,1603136926,66,22.89,-109.91,91.99,4.70
2,Mar del Plata,98,AR,1603137015,39,-38.00,-57.56,62.60,14.99
3,Puntarenas,75,CR,1603137665,74,9.98,-84.84,78.80,11.41
4,Jamestown,0,SH,1603137124,97,-15.94,-5.72,61.99,21.68
...,...,...,...,...,...,...,...,...,...
520,Domingos Martins,75,BR,1603138310,78,-20.36,-40.66,75.20,6.93
521,Dawson Creek,100,CA,1603138310,89,55.77,-120.24,28.89,7.43
522,Ciudad Bolivar,90,VE,1603138310,39,8.12,-63.55,94.55,5.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
weights = cities_df['Humidity'].astype(float)

cities_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

cities_fig.add_layer(heat_layer)

# Show the heatmap
cities_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Drop any null values
cities_df = cities_df.dropna()


# Setup variables for searching for the perfect vacation spot
low_temp = 70
high_temp = 80
max_wind_speed = 10
cloudiness = 0
humidity = 30

# Create a DataFrame with only cities that have our ideal weather
vacation_cities_df = cities_df.loc[cities_df['Wind Speed'] < max_wind_speed]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Cloudiness'] ==  cloudiness]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] > low_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] < high_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Humidity'] < humidity]
vacation_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
142,Hwange,0,ZW,1603137795,19,-18.37,26.50,77.49,3.00
172,Hīt,0,IQ,1603137857,23,33.64,42.83,70.68,3.18
283,Riyadh,0,SA,1603137444,22,24.69,46.72,73.40,4.70
306,Vryburg,0,ZA,1603138049,19,-26.96,24.73,74.77,7.76
321,Menongue,0,AO,1603138051,20,-14.66,17.69,71.29,3.65
372,Pūndri,0,IN,1603138116,22,29.75,76.55,70.21,3.51
427,General Roca,0,AR,1603138142,21,-39.03,-67.58,71.60,3.36
478,Fazilka,0,IN,1603138247,21,30.40,74.03,71.40,4.36
514,Monze,0,ZM,1603138310,21,-16.28,27.48,72.61,4.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create a new dataframe and add a Hotel Name column
hotel_df = pd.DataFrame(vacation_cities_df,
                      columns=['City','Country','Lat','Lng','Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
142,Hwange,ZW,-18.37,26.50,NaN
172,Hīt,IQ,33.64,42.83,NaN
283,Riyadh,SA,24.69,46.72,NaN
306,Vryburg,ZA,-26.96,24.73,NaN
321,Menongue,AO,-14.66,17.69,NaN
372,Pūndri,IN,29.75,76.55,NaN
427,General Roca,AR,-39.03,-67.58,NaN
478,Fazilka,IN,30.40,74.03,NaN
514,Monze,ZM,-16.28,27.48,NaN


In [7]:
# find the closest hotel for all of our possible vacation cities

# Define the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through possible vacation cities
for index, row in hotel_df.iterrows():
    
    # Set a variable for the latitude and Longitude of our city
    city_lat_lng = f"{row['Lat']},{row['Lng']}"
    
    # Define the api call parameters
    params = {
               "location": city_lat_lng,  
               "rankby": "distance",
               "types": "lodging",
               "key": g_key
             }
    # Output that we are retrieving information for our city
    print(f"Retrieving Results for Index {index}: {row['City']}, {row['Country']}.")
   
    # assemble url and make API request
    response_json = requests.get(base_url, params=params).json()
    
    # extract results
    results = response_json['results']

    # If we got a hotel add it to the DataFrame
    # if we don't get a hotel then output a message that we are skipping it
    try:
        # Output the closet hotel
        print(f"Closest hotel is {results[0]['name']}.")
        
        # Update the data frame with the hotel name
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------------------------------------------")

hotel_df   

Retrieving Results for Index 142: Hwange, ZW.
Closest hotel is Hwange Colliery Company Guest Lodge.
-----------------------------------------------------
Retrieving Results for Index 172: Hīt, IQ.
Closest hotel is صالون زيرو.
-----------------------------------------------------
Retrieving Results for Index 283: Riyadh, SA.
Closest hotel is عمارة الشلهوب.
-----------------------------------------------------
Retrieving Results for Index 306: Vryburg, ZA.
Closest hotel is Times Premier Lodge.
-----------------------------------------------------
Retrieving Results for Index 321: Menongue, AO.
Closest hotel is Complexo Turístico Reencontro.
-----------------------------------------------------
Retrieving Results for Index 372: Pūndri, IN.
Closest hotel is गर्ग फार्मेसी.
-----------------------------------------------------
Retrieving Results for Index 427: General Roca, AR.
Closest hotel is Don Bosco.
-----------------------------------------------------
Retrieving Results for Index 478:

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
142,Hwange,ZW,-18.37,26.50,Hwange Colliery Company Guest Lodge
172,Hīt,IQ,33.64,42.83,صالون زيرو
283,Riyadh,SA,24.69,46.72,عمارة الشلهوب
306,Vryburg,ZA,-26.96,24.73,Times Premier Lodge
321,Menongue,AO,-14.66,17.69,Complexo Turístico Reencontro
372,Pūndri,IN,29.75,76.55,गर्ग फार्मेसी
427,General Roca,AR,-39.03,-67.58,Don Bosco
478,Fazilka,IN,30.40,74.03,Hanish Kumar gift center
514,Monze,ZM,-16.28,27.48,Sera's Lodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
cities_fig.add_layer(markers)

# Display Map
cities_fig

Figure(layout=FigureLayout(height='420px'))